# Ejercicio 4: Modelo Probabilístico

## Objetivo de la práctica
- Comprender los componentes del modelo vectorial mediante cálculos manuales y observación directa.
- Aplicar el modelo de espacio vectorial con TF-IDF para recuperar documentos relevantes.
- Comparar la recuperación con BM25 frente a TF-IDF.
- Analizar visualmente las diferencias entre los modelos.
- Evaluar si los rankings generados son consistentes con lo que considerarías documentos relevantes.

## Parte 0: Carga del Corpus

In [1]:
from sklearn.datasets import fetch_20newsgroups

newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
newsgroupsdocs = newsgroups.data

In [2]:
#tipo de newsgroupsdocs
print(type(newsgroupsdocs))

<class 'list'>


In [3]:
#ver un documento
print(newsgroupsdocs[0])



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!




In [4]:
#peso de newsgroupsdocs
print(newsgroupsdocs.__sizeof__())

150808


In [5]:
#vectorizando newsgroupsdocs 
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
#tamaño del vector
X = vectorizer.fit_transform(newsgroupsdocs)
print(X.shape)
corpus_vect=vectorizer.transform(newsgroupsdocs)
corpus_vect.toarray()
#tamaño del vector
print(corpus_vect.shape)


(18846, 134410)
(18846, 134410)


In [6]:
# query chicken
query = ["ÿhooked"]
#vectorizando query
query_vect = vectorizer.transform(query)
#valor de la query
#imprime segun  el idf el vector
print(query_vect.toarray())
#similaridad entre query y corpus
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(query_vect, corpus_vect)
#similaridad entre query y corpus
print(similarity)

[[0. 0. 0. ... 0. 0. 1.]]
[[0. 0. 0. ... 0. 0. 0.]]


In [7]:
vectorizer.get_feature_names_out()

array(['00', '000', '0000', ..., '³ation', 'ýé', 'ÿhooked'],
      shape=(134410,), dtype=object)

In [9]:
from nltk.corpus import stopwords, words

In [ ]:
import nltk
# Descargar recursos necesarios
#natural language toolkit
nltk.download('stopwords')
nltk.download('words')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ELI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\ELI\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [12]:
import re

In [13]:
# Corpus de palabras válidas en inglés
palabras_validas = set(words.words())
stop_words = set(stopwords.words('english'))

# Cargar corpus
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
docs = newsgroups.data

# Función de limpieza
def limpiar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r'[^a-z\s]', '', texto)  # solo letras y espacios
    palabras = texto.split()
    palabras_filtradas = [
        palabra for palabra in palabras
        if palabra in palabras_validas and palabra not in stop_words and len(palabra) > 2
    ]
    return ' '.join(palabras_filtradas)

# Aplicar limpieza al corpus
corpus_limpio = [limpiar_texto(doc) for doc in docs]

In [14]:
# Vectorizar corpus limpio
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus_limpio)

# Tamaño del corpus vectorizado
print(X.shape)

(18846, 23146)


In [15]:
print(type(X))

<class 'scipy.sparse._csr.csr_matrix'>


In [16]:
# Vectorizar query también limpiándola
query_clean = ["sure"]
query_limpia = [limpiar_texto(q) for q in query_clean]
query_vectorizado = vectorizer.transform(query_limpia)
print(query_vectorizado.toarray())

[[0. 0. 0. ... 0. 0. 0.]]


In [17]:
# Similaridad
similarity_clean = cosine_similarity(query_vectorizado, X)
print(similarity_clean)
print(len(corpus_limpio))  # o len(newsgroupsdocs)


[[0.08660636 0.         0.         ... 0.         0.         0.        ]]
18846


## Parte 1: Cálculo de TF, DF, IDF y TF-IDF

### Actividad 
1. Utiliza el corpus cargado.
2. Construye la matriz de términos (TF), y calcula la frecuencia de documentos (DF)
3. Calcula TF-IDF utilizando sklearn.
4. Visualiza los valores en un DataFrame para analizar las diferencias entre los términos.

In [18]:
import pandas as pd


In [19]:
import numpy as np

In [20]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# Calcular la matriz TF-IDF
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X)
#Obtener la matriz TF (frecuencia de términos)
X_tf = CountVectorizer().fit_transform(corpus_limpio)
# Obtener la matriz IDF (importancia global de cada término)
idf_values = tfidf_transformer.idf_
# Obtener la matriz TF-IDF
X_tfidf = tfidf_transformer.transform(X_tf)

# Convertir a DataFrames para visualización
tf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
idf_values = tfidf_transformer.idf_
idf_df = pd.DataFrame([idf_values], columns=vectorizer.get_feature_names_out())
tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=vectorizer.get_feature_names_out())

# Mostrar la matriz TF (frecuencia de términos), IDF (importancia global de cada término) y TF-IDF
print("Matriz TF (frecuencia de términos):")
print(tf_df.iloc[:, :])  # primeras 20 columnas
print("\nIDF (importancia global de cada término):")
print(idf_df.iloc[:, :])
print("Matriz TF-IDF:")
print(X_tfidf.toarray())



Matriz TF (frecuencia de términos):
       aam  abandon  abandoned  abandonment  abate  abatement  abbas  abbey  \
0      0.0      0.0        0.0          0.0    0.0        0.0    0.0    0.0   
1      0.0      0.0        0.0          0.0    0.0        0.0    0.0    0.0   
2      0.0      0.0        0.0          0.0    0.0        0.0    0.0    0.0   
3      0.0      0.0        0.0          0.0    0.0        0.0    0.0    0.0   
4      0.0      0.0        0.0          0.0    0.0        0.0    0.0    0.0   
...    ...      ...        ...          ...    ...        ...    ...    ...   
18841  0.0      0.0        0.0          0.0    0.0        0.0    0.0    0.0   
18842  0.0      0.0        0.0          0.0    0.0        0.0    0.0    0.0   
18843  0.0      0.0        0.0          0.0    0.0        0.0    0.0    0.0   
18844  0.0      0.0        0.0          0.0    0.0        0.0    0.0    0.0   
18845  0.0      0.0        0.0          0.0    0.0        0.0    0.0    0.0   

       abbot  a

In [21]:
vectorizer = CountVectorizer()
tf_matrix = vectorizer.fit_transform(corpus_limpio)
# Obtener el vocabulario (términos)
vocabulario = vectorizer.get_feature_names_out()

# Mostrar dimensiones de la matriz TF
print("Dimensión de la matriz TF (documentos x términos):", tf_matrix.shape)

Dimensión de la matriz TF (documentos x términos): (18846, 23146)


In [22]:
# Mostrar TF de un documento de ejemplo
print("TF del documento 0 (frecuencia de cada término):")
print(tf_matrix[12].toarray())

TF del documento 0 (frecuencia de cada término):
[[0 0 1 ... 0 0 0]]


In [23]:
# Calcular DF: cuántos documentos contienen cada término
df_vector = np.asarray((tf_matrix > 0).sum(axis=0)).ravel()

# Mostrar algunos ejemplos de términos y su DF
for termino, df in zip(vocabulario[:], df_vector[:]):
    print(f"Término: {termino} - DF: {df}")

Término: aam - DF: 1
Término: abandon - DF: 32
Término: abandoned - DF: 39
Término: abandonment - DF: 1
Término: abate - DF: 1
Término: abatement - DF: 1
Término: abbas - DF: 1
Término: abbey - DF: 5
Término: abbot - DF: 2
Término: abbreviation - DF: 9
Término: abdicate - DF: 1
Término: abdication - DF: 1
Término: abdomen - DF: 6
Término: abdominal - DF: 9
Término: abduct - DF: 2
Término: abduction - DF: 3
Término: aberrant - DF: 8
Término: aberration - DF: 6
Término: abhor - DF: 5
Término: abhorrence - DF: 1
Término: abhorrent - DF: 4
Término: abide - DF: 9
Término: abiding - DF: 18
Término: ability - DF: 259
Término: abiogenesis - DF: 4
Término: abject - DF: 3
Término: ablaze - DF: 1
Término: able - DF: 987
Término: ably - DF: 4
Término: abnormal - DF: 9
Término: abnormally - DF: 2
Término: aboard - DF: 14
Término: abode - DF: 9
Término: abolish - DF: 18
Término: abolishment - DF: 2
Término: abolition - DF: 5
Término: abolitionist - DF: 2
Término: abominable - DF: 7
Término: abominat

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Construcción de la matriz TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(corpus_limpio)

# Obtener los términos del vocabulario
terminos = vectorizer.get_feature_names_out()

# Crear un DataFrame con los valores TF-IDF 
df_tfidf = pd.DataFrame(tfidf_matrix[:].toarray(), columns=terminos)

# Visualizar las primeras filas del DataFrame
print(df_tfidf.head())

# Visualizar los primeros términos y su peso en los primeros documentos
print(df_tfidf.T.head())


   aam  abandon  abandoned  abandonment  abate  abatement  abbas  abbey  \
0  0.0      0.0        0.0          0.0    0.0        0.0    0.0    0.0   
1  0.0      0.0        0.0          0.0    0.0        0.0    0.0    0.0   
2  0.0      0.0        0.0          0.0    0.0        0.0    0.0    0.0   
3  0.0      0.0        0.0          0.0    0.0        0.0    0.0    0.0   
4  0.0      0.0        0.0          0.0    0.0        0.0    0.0    0.0   

   abbot  abbreviation  ...  zooid  zoological  zoologist  zoology  zoom  \
0    0.0           0.0  ...    0.0         0.0        0.0      0.0   0.0   
1    0.0           0.0  ...    0.0         0.0        0.0      0.0   0.0   
2    0.0           0.0  ...    0.0         0.0        0.0      0.0   0.0   
3    0.0           0.0  ...    0.0         0.0        0.0      0.0   0.0   
4    0.0           0.0  ...    0.0         0.0        0.0      0.0   0.0   

   zoophile  zoophilia  zorro  zounds  zygon  
0       0.0        0.0    0.0     0.0    0.0 

## Parte 2: Ranking de documentos usando TF-IDF

### Actividad 

1. Dada una consulta, construye el vector de consulta
2. Calcula la similitud coseno entre la consulta y cada documento usando los vectores TF-IDF
3. Genera un ranking de los documentos ordenados por relevancia.
4. Muestra los resultados en una tabla.

In [44]:
# Vectorizar query también limpiándola
query_clean = ["aam"]
query_limpia = [limpiar_texto(q) for q in query_clean]
query_vectorizado = vectorizer.transform(query_limpia)
print(query_vectorizado.toarray())


[[1. 0. 0. ... 0. 0. 0.]]


In [ ]:
#distancia coseno entre query y corpus limpio
from sklearn.metrics.pairwise import cosine_similarity

similarity_clean = cosine_similarity(query_vectorizado, tfidf_matrix)
print("Similitud entre la query y el corpus limpio:")
print(similarity_clean)
print("Dimensiones del vector de similitud:", similarity_clean.shape)



Similitud entre la query y el corpus limpio:
[[0. 0. 0. ... 0. 0. 0.]]
Dimensiones del vector de similitud: (1, 18846)


In [ ]:
#ordenar los resultados de similitud
import numpy as np
# Obtener los índices de los documentos ordenados por similitud
indices_ordenados = np.argsort(similarity_clean[0])[::-1]
# Mostrar los documentos más similares
print("Documentos más similares a la query:")
for idx in indices_ordenados[:]: 
    print(f"Documento {idx}: Similaridad = {similarity_clean[0][idx]}")

Documentos más similares a la query:
Documento 13270: Similaridad = 0.026612314770617474
Documento 18833: Similaridad = 0.0
Documento 18832: Similaridad = 0.0
Documento 31: Similaridad = 0.0
Documento 18816: Similaridad = 0.0
Documento 15: Similaridad = 0.0
Documento 16: Similaridad = 0.0
Documento 18831: Similaridad = 0.0
Documento 0: Similaridad = 0.0
Documento 18845: Similaridad = 0.0
Documento 18844: Similaridad = 0.0
Documento 18843: Similaridad = 0.0
Documento 18842: Similaridad = 0.0
Documento 18841: Similaridad = 0.0
Documento 18840: Similaridad = 0.0
Documento 18835: Similaridad = 0.0
Documento 30: Similaridad = 0.0
Documento 29: Similaridad = 0.0
Documento 28: Similaridad = 0.0
Documento 27: Similaridad = 0.0
Documento 26: Similaridad = 0.0
Documento 25: Similaridad = 0.0
Documento 24: Similaridad = 0.0
Documento 23: Similaridad = 0.0
Documento 22: Similaridad = 0.0
Documento 21: Similaridad = 0.0
Documento 20: Similaridad = 0.0
Documento 19: Similaridad = 0.0
Documento 18: S

## Parte 3: Ranking con BM25

### Actividad 

1. Implementa un sistema de recuperación usando el modelo BM25.
2. Usa la misma consulta del ejercicio anterior.
3. Calcula el score BM25 para cada documento y genera un ranking.
4. Compara manualmente con el ranking de TF-IDF.

## Parte 4: Comparación visual entre TF-IDF y BM25

### Actividad 

1. Utiliza un gráfico de barras para visualizar los scores obtenidos por cada documento según TF-IDF y BM25.
2. Compara los rankings visualmente.
3. Identifica: ¿Qué documentos obtienen scores más altos en un modelo que en otro?
4. Sugiere: ¿A qué se podría deber esta diferencia?

## Parte 5: Evaluación con consulta relevante

### Actividad 

1. Elige una consulta y define qué documentos del corpus deberían considerarse relevantes.
2. Evalúa Precision@3 o MAP para los rankings generados con TF-IDF y BM25.
3. Responde: ¿Cuál modelo da mejores resultados respecto a tu criterio de relevancia?